In [8]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import numpy as np
from scipy import optimize
import numba as nb


from mwc_class import getMVCs
from mwc_functions import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
gerdata = getMVCs('germany.csv', save_results=False)
results = gerdata.run_pipeline()


In [10]:
transformed_dataframe = results.get("transformed_dataframe")
parties_in_year = results.get("parties_in_year")
totalseats_in_year = results.get("totalseats_in_year")
coalition_dict = results.get("coalition_dict")
winning_coal_dict = results.get("winning_coal_dict")
minimal_winning_coalitions = results.get("minimal_winning_coalitions")
maximal_losing_coalitions = results.get("maximal_losing_coalitions")
unique_tying_coalitions = results.get("unique_tying_coalitions")

In [11]:
def create_all_year_dfs(winning_coal_dict, parties_in_year):
    ##create dataframes fro every year from coalition dict and indicate whether the coal was winning 
    ##stores all dataframes in a dict with key= year
    all_year_dfs = {}

    for (year, coalition), is_winning in winning_coal_dict.items():
        if year not in all_year_dfs:#make new df if not there already
            all_year_dfs[year] = pd.DataFrame(columns=parties_in_year[year] + ['Winning']) #make column for every party and one to indicate if winning

        # Rows
        row = {party: 1 if party in coalition.split('+') else 0 for party in parties_in_year[year]} #logic: write 1 in cell if party is member of the coalition, 0 if not
        row['Winning'] = is_winning # write 1 in winning column if coal was winning

        # Add rows to df
        all_year_dfs[year].loc[coalition] = row

    return all_year_dfs

In [12]:
from optimization_functions import *
all_year_dfs = create_all_year_dfs(winning_coal_dict, parties_in_year)

Example

In [13]:
year = '2002' 
df_for_year = all_year_dfs.get(year)
print(df_for_year)


                         CDU  CSU  FDP  G -  PDS  SPD  Winning
                           0    0    0    0    0    0        0
CDU                        1    0    0    0    0    0        0
CSU                        0    1    0    0    0    0        0
FDP                        0    0    1    0    0    0        0
G -                        0    0    0    1    0    0        0
...                      ...  ...  ...  ...  ...  ...      ...
CDU+CSU+FDP+PDS+SPD        1    1    1    0    1    1        1
CDU+CSU+G -+PDS+SPD        1    1    0    1    1    1        1
CDU+FDP+G -+PDS+SPD        1    0    1    1    1    1        1
CSU+FDP+G -+PDS+SPD        0    1    1    1    1    1        1
CDU+CSU+FDP+G -+PDS+SPD    1    1    1    1    1    1        1

[64 rows x 7 columns]


apparently the next cell is not good practice since python does not like global variables. 
Later everything will be written in terms of dicts, for now, this will do.

In [14]:
for year, df in all_year_dfs.items():
    globals()[f'df_of_{year}'] = df

now it gets interesting

In [15]:
def generate_constraints(df):
    # Split the DataFrame into winning and losing coalitions
    winning_coalitions = df[df['Winning'] == 1]
    losing_coalitions = df[df['Winning'] == 0]

    constraints = []

    # Iterate over each winning and losing coalition pair to generate constraints
    for winning_index, winning_row in winning_coalitions.iterrows(): #for 
        for losing_index, losing_row in losing_coalitions.iterrows():
            # Sum weights for winning coalition
            winning_sum = "+".join([f'w_{party}' for party in df.columns[:-1] if winning_row[party] == 1])

            # Sum weights for losing coalition
            losing_sum = "+".join([f'w_{party}' for party in df.columns[:-1] if losing_row[party] == 1])

            # Create the constraint string
            if winning_sum and losing_sum:  # Ensure both sums are non-empty
                constraint = f'({winning_sum}) - ({losing_sum}) > 0'
                constraints.append(constraint)

    return constraints

In [19]:
def generate_constraints_df(df):
    '''outer-loop iterates over each winning coalition and inner-loop iterates over each losing coalition '''
    '''If a party is in the winning coal but not in the losing it gets 1 as constraint.
    If the party is only in the losing coal, it gets -1.
    If a party is in both or neither, it gets 0.'''
    # Initialize an empty DataFrame for constraints
    parties = df.columns[:-1]  # create partyweights
    constraints_df = pd.DataFrame(columns=parties) #create new empty df with weights as columns  
    
    winning_coalitions = df[df['Winning'] == 1] # Split original DataFrame into winning and losing coalitions
    losing_coalitions = df[df['Winning'] == 0]

    # Generate constraints
    constraint_index = 0 #each win-lose pair creates one constraint
    for _, winning_row in winning_coalitions.iterrows(): #outerloop with the winning coals
        for _, losing_row in losing_coalitions.iterrows(): #inner loop with the losing coals
            # Create a row for the constraint
            constraint_row = {}
            for party in parties: #calculate difference between the winning and the losing coal
                constraint_row[party] = winning_row[party] - losing_row[party] #if party is in winning coal it is assigned a 1, if party is in losing coal it is assigned -1, 0 o/w. 

            # Add the constraint to the DataFrame
            constraints_df.loc[constraint_index] = constraint_row
            constraint_index += 1

    return constraints_df

In [16]:
constraints = generate_constraints(df_for_year)

In [20]:
# Example usage:
constraints_df = generate_constraints_df(df_for_year)